In [15]:
from rdflib import Graph, Literal
from rdflib.namespace import RDF, RDFS, SKOS
import rdflib
import os
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)
sparql.setTimeout(20)
schema = rdflib.Namespace("http://schema.org/")
owl = rdflib.Namespace("http://www.w3.org/2002/07/owl#")
ocs = rdflib.Namespace("https://w3id.org/ocs/ont/")
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
from src.helpers import get_labels
from src.helpers import get_o

In [3]:
language = "pl"
directory = r"../OpenCS/ontology/core/00/"

In [5]:
no_entities_added = 0
g = Graph()
g.bind("ocs", ocs)
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        new_node = Graph()
        new_node.parse(f)
        g += new_node
        no_entities_added +=1
        

In [6]:
no_entities_added

999

In [7]:
enitites_to_find = pd.DataFrame([(str(o), s) for s, p, o in g.triples((None, SKOS.closeMatch, None))], columns=["uri", "filename"])
enitites_to_find

,uri,filename
0,http://dbpedia.org/resource/Computer_science,https://w3id.org/ocs/ont/C1
1,http://dbpedia.org/resource/Algorithm,https://w3id.org/ocs/ont/C10
2,http://dbpedia.org/resource/Embedded_system,https://w3id.org/ocs/ont/C11
3,http://dbpedia.org/resource/Reconfigurability,https://w3id.org/ocs/ont/C112
4,http://dbpedia.org/resource/REEM,https://w3id.org/ocs/ont/C114
...,...,...
584,http://dbpedia.org/resource/Smart_key,https://w3id.org/ocs/ont/C990
585,http://dbpedia.org/resource/Survivability,https://w3id.org/ocs/ont/C991
586,http://dbpedia.org/resource/Microsoft_Talisman,https://w3id.org/ocs/ont/C992
587,http://dbpedia.org/resource/Monitor_unit,https://w3id.org/ocs/ont/C993


In [8]:
len(enitites_to_find)

589

In [10]:
names = ["(dbr:" + uri.split("/").pop() + ")" for uri in enitites_to_find['uri']]
names = [uri.split("/").pop()
             .replace("*", "\*")
             .replace("(", "\(")
             .replace(")", "\)")
             .replace(",", "\,")
             .replace("'", "\\'")
             .replace("-", "\-") for uri in enitites_to_find['uri']]
names = ["(dbr:" + uri + ")" for uri in names]

In [12]:
len(names)

589

In [16]:
labels = []
for idx_start in range(0, len(names), 10):
    idx_end = idx_start + 10
    labels.append(get_labels(names[idx_start:idx_end], language))
labels_df = pd.concat(labels)

In [18]:
len(labels_df)

184

In [19]:
changed_enities = labels_df.merge(enitites_to_find, on = "uri", how="left")
changed_enities_names = [str(entity) for entity in changed_enities['filename']]

In [20]:
changed_enities

,uri,label,filename
0,http://dbpedia.org/resource/Algorithm,Algorytm,https://w3id.org/ocs/ont/C10
1,http://dbpedia.org/resource/Computer_science,Informatyka,https://w3id.org/ocs/ont/C1
2,http://dbpedia.org/resource/Embedded_system,System wbudowany,https://w3id.org/ocs/ont/C11
3,http://dbpedia.org/resource/Cellular_automaton,Automat komórkowy,https://w3id.org/ocs/ont/C122
4,http://dbpedia.org/resource/Digital_filter,Filtr cyfrowy,https://w3id.org/ocs/ont/C123
...,...,...,...
179,http://dbpedia.org/resource/Timeout_(computing),Timeout (telekomunikacja),https://w3id.org/ocs/ont/C965
180,http://dbpedia.org/resource/Tf–idf,TFIDF,https://w3id.org/ocs/ont/C972
181,http://dbpedia.org/resource/Voxel,Woksel,https://w3id.org/ocs/ont/C969
182,http://dbpedia.org/resource/No-fly_zone,Strefa zakazu lotów,https://w3id.org/ocs/ont/C981


In [21]:
graph_triples ={}
for s, p, o in g.triples((None, None, None)):
    if str(s) in changed_enities_names:
        entity_name = s.split("/").pop()
        if entity_name not in graph_triples.keys():
            graph_triples[entity_name] = []
            #graph_triples[entity_name].bind("ocs", ocs)    
        graph_triples[entity_name].append((s, p, o))

In [22]:
graph_triples["C1"]

[(rdflib.term.URIRef('https://w3id.org/ocs/ont/C1'),
  rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#related'),
  rdflib.term.URIRef('https://w3id.org/ocs/ont/C19')),
 (rdflib.term.URIRef('https://w3id.org/ocs/ont/C1'),
  rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#related'),
  rdflib.term.URIRef('https://w3id.org/ocs/ont/C14')),
 (rdflib.term.URIRef('https://w3id.org/ocs/ont/C1'),
  rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#related'),
  rdflib.term.URIRef('https://w3id.org/ocs/ont/C197')),
 (rdflib.term.URIRef('https://w3id.org/ocs/ont/C1'),
  rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#related'),
  rdflib.term.URIRef('https://w3id.org/ocs/ont/C8')),
 (rdflib.term.URIRef('https://w3id.org/ocs/ont/C1'),
  rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#closeMatch'),
  rdflib.term.URIRef('http://dbpedia.org/resource/Computer_science')),
 (rdflib.term.URIRef('https://w3id.org/ocs/ont/C1'),
  rdflib.term.URIRef('http://www.w3.org/200

In [23]:
graph = {}
for key, value in graph_triples.items():
    val = value.copy()
    val.sort(key = get_o)
    g = Graph()
    g.bind("ocs", ocs)
    for triple in value:
        g.add((s, p, o))
    graph[key] = g

In [24]:
val = value.copy()
val.sort(key = get_o)
g = Graph()
g.bind("ocs", ocs)
for triple in value:
    g.add((s, p, o))

In [25]:
g

<Graph identifier=Nad3ec4622c89423ba0d51d5913c1995a (<class 'rdflib.graph.Graph'>)>

In [26]:
for s, p, o in graph["C401"].triples((None, None, None)):
    print(s,p,o)

https://w3id.org/ocs/ont/C58 http://www.w3.org/2004/02/skos/core#related https://w3id.org/ocs/ont/C40348


In [36]:
for idx, row in changed_enities.iterrows():
    uri = row['filename'].split("/").pop()
    label = Literal(row['label'], lang = language)
    graph[uri].add((rdflib.term.URIRef(ocs[uri]), SKOS.prefLabel, label))

In [38]:
graph['C401'].serialize(destination = "test.ttl")

<Graph identifier=N5d3a263efcba44598459b9ab475b9102 (<class 'rdflib.graph.Graph'>)>

In [21]:
for key, value in graph.items():
    value.serialize(destination=directory+key+".ttl")

In [20]:
directory

'../OpenCS/ontology/core/00/'